# Telecom Churn case study using PCA and Ensemble

### Problem Statement
In the telecom industry, customers are able to choose from multiple service providers and actively switch from one operator to another. In this highly competitive market, the telecommunications industry experiences an average of 15-25% annual churn rate. Given the fact that it costs 5-10 times more to acquire a new customer than to retain an existing one, customer retention has now become even more important than customer acquisition.

For many incumbent operators, retaining high profitable customers is the number one business
goal. To reduce customer churn, telecom companies need to predict which customers are at high risk of churn. In this project, you will analyze customer-level data of a leading telecom firm, build predictive models to identify customers at high risk of churn.

In this competition, your goal is to build a machine learning model that is able to predict churning customers based on the features provided for their usage.

### Goal
It is your job to predict if a customer will churn, given the ~170 columns containing customer behavior, usage patterns, payment patterns, and other features that might be relevant. Your target variable is "churn_probability"
Note: Make sure your accuracy is greater than the sample submission that is present in the leaderboard

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
import warnings

from sklearn import metrics
from sklearn.decomposition import IncrementalPCA
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.preprocessing import MinMaxScaler

# to ignore the warnings
warnings.filterwarnings('ignore')

## Step 1. Reading, Understanding and Visualizing the data

In [2]:
telecom_train_df = pd.read_csv('train (1).csv')
telecom_test_df = pd.read_csv('test (1).csv')

In [3]:
telecom_train_df.head()

,id,circle_id,loc_og_t2o_mou,std_og_t2o_mou,loc_ic_t2o_mou,last_date_of_month_6,last_date_of_month_7,last_date_of_month_8,arpu_6,arpu_7,...,sachet_3g_7,sachet_3g_8,fb_user_6,fb_user_7,fb_user_8,aon,aug_vbc_3g,jul_vbc_3g,jun_vbc_3g,churn_probability
0,0,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,31.277,87.009,...,0,0,NaN,NaN,NaN,1958,0.0,0.0,0.0,0
1,1,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,0.000,122.787,...,0,0,NaN,1.0,NaN,710,0.0,0.0,0.0,0
2,2,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,60.806,103.176,...,0,0,NaN,NaN,NaN,882,0.0,0.0,0.0,0
3,3,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,156.362,205.260,...,0,0,NaN,NaN,NaN,982,0.0,0.0,0.0,0
4,4,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,240.708,128.191,...,1,0,1.0,1.0,1.0,647,0.0,0.0,0.0,0


In [4]:
telecom_test_df.head()

,id,circle_id,loc_og_t2o_mou,std_og_t2o_mou,loc_ic_t2o_mou,last_date_of_month_6,last_date_of_month_7,last_date_of_month_8,arpu_6,arpu_7,...,sachet_3g_6,sachet_3g_7,sachet_3g_8,fb_user_6,fb_user_7,fb_user_8,aon,aug_vbc_3g,jul_vbc_3g,jun_vbc_3g
0,69999,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,91.882,65.330,...,0,0,0,NaN,NaN,NaN,1692,0.00,0.00,0.00
1,70000,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,414.168,515.568,...,0,0,0,NaN,NaN,NaN,2533,0.00,0.00,0.00
2,70001,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,329.844,434.884,...,0,0,0,NaN,NaN,NaN,277,525.61,758.41,241.84
3,70002,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,43.550,171.390,...,0,0,0,NaN,NaN,NaN,1244,0.00,0.00,0.00
4,70003,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,306.854,406.289,...,0,0,0,NaN,NaN,NaN,462,0.00,0.00,0.00


In [5]:
telecom_train_df.shape

(69999, 172)

In [6]:
telecom_test_df.shape

(30000, 171)

In [7]:
telecom_train_df.columns

Index(['id', 'circle_id', 'loc_og_t2o_mou', 'std_og_t2o_mou', 'loc_ic_t2o_mou',
       'last_date_of_month_6', 'last_date_of_month_7', 'last_date_of_month_8',
       'arpu_6', 'arpu_7',
       ...
       'sachet_3g_7', 'sachet_3g_8', 'fb_user_6', 'fb_user_7', 'fb_user_8',
       'aon', 'aug_vbc_3g', 'jul_vbc_3g', 'jun_vbc_3g', 'churn_probability'],
      dtype='object', length=172)

In [8]:
telecom_train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69999 entries, 0 to 69998
Columns: 172 entries, id to churn_probability
dtypes: float64(135), int64(28), object(9)
memory usage: 91.9+ MB


In [9]:
telecom_train_df.describe()

,id,circle_id,loc_og_t2o_mou,std_og_t2o_mou,loc_ic_t2o_mou,arpu_6,arpu_7,arpu_8,onnet_mou_6,onnet_mou_7,...,sachet_3g_7,sachet_3g_8,fb_user_6,fb_user_7,fb_user_8,aon,aug_vbc_3g,jul_vbc_3g,jun_vbc_3g,churn_probability
count,69999.000000,69999.0,69297.0,69297.0,69297.0,69999.000000,69999.000000,69999.000000,67231.000000,67312.000000,...,69999.000000,69999.000000,17568.000000,17865.000000,18417.000000,69999.000000,69999.000000,69999.000000,69999.00000,69999.000000
mean,34999.000000,109.0,0.0,0.0,0.0,283.134365,278.185912,278.858826,133.153275,133.894438,...,0.081444,0.085487,0.916325,0.909544,0.890319,1220.639709,68.108597,65.935830,60.07674,0.101887
std,20207.115084,0.0,0.0,0.0,0.0,334.213918,344.366927,351.924315,299.963093,311.277193,...,0.634547,0.680035,0.276907,0.286842,0.312501,952.426321,269.328659,267.899034,257.22681,0.302502
min,0.000000,109.0,0.0,0.0,0.0,-2258.709000,-1289.715000,-945.808000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,180.000000,0.000000,0.000000,0.00000,0.000000
25%,17499.500000,109.0,0.0,0.0,0.0,93.581000,86.714000,84.095000,7.410000,6.675000,...,0.000000,0.000000,1.000000,1.000000,1.000000,468.000000,0.000000,0.000000,0.00000,0.000000
50%,34999.000000,109.0,0.0,0.0,0.0,197.484000,191.588000,192.234000,34.110000,32.280000,...,0.000000,0.000000,1.000000,1.000000,1.000000,868.000000,0.000000,0.000000,0.00000,0.000000
75%,52498.500000,109.0,0.0,0.0,0.0,370.791000,365.369500,369.909000,119.390000,115.837500,...,0.000000,0.000000,1.000000,1.000000,1.000000,1813.000000,0.000000,0.000000,0.00000,0.000000
max,69998.000000,109.0,0.0,0.0,0.0,27731.088000,35145.834000,33543.624000,7376.710000,8157.780000,...,33.000000,41.000000,1.000000,1.000000,1.000000,4337.000000,12916.220000,9165.600000,11166.21000,1.000000


In [10]:
telecom_train_df.isna().sum()

id                     0
circle_id              0
loc_og_t2o_mou       702
std_og_t2o_mou       702
loc_ic_t2o_mou       702
                    ... 
aon                    0
aug_vbc_3g             0
jul_vbc_3g             0
jun_vbc_3g             0
churn_probability      0
Length: 172, dtype: int64

In [11]:
telecom_train_df.dtypes

id                     int64
circle_id              int64
loc_og_t2o_mou       float64
std_og_t2o_mou       float64
loc_ic_t2o_mou       float64
                      ...   
aon                    int64
aug_vbc_3g           float64
jul_vbc_3g           float64
jun_vbc_3g           float64
churn_probability      int64
Length: 172, dtype: object

In [12]:
# Columns which we have to impute as Zero as thy should not be dropped based on missing values becuase they are important.
rech_cols_to_impute = [x for x in telecom_train_df.columns if 'rech' in x and 
                       'count' not in x and 'date' not in x and 'num' not in x]
telecom_train_df[rech_cols_to_impute] = telecom_train_df[rech_cols_to_impute].apply(lambda x: x.fillna(0))
telecom_test_df[rech_cols_to_impute] = telecom_test_df[rech_cols_to_impute].apply(lambda x: x.fillna(0))


### Dropping rows for both train and test datasets
 - Rows which are empty

In [13]:
telecom_train_df.dropna(axis=0, how='all', inplace=True)
telecom_test_df.dropna(axis=0, how='all', inplace=True)

In [14]:
print(telecom_train_df.shape)
print(telecom_test_df.shape)

(69999, 172)
(30000, 171)


### Dropping columns for both train and test datasets

    - Not needed columns like Id, last_date_of_month_6 etc.
    - Columns having more than 70% values as null

In [15]:
# For us dates doesn't matter as long as customer is doing a recharge.
cols_to_delete = ['id', 'last_date_of_month_6', 'last_date_of_month_7', 'last_date_of_month_8',
                  'date_of_last_rech_data_6', 'date_of_last_rech_data_7', 'date_of_last_rech_data_8',
                  'date_of_last_rech_6', 'date_of_last_rech_7', 'date_of_last_rech_8'
                 ]
telecom_train_df.drop(cols_to_delete, axis=1, inplace=True)
telecom_test_df.drop(cols_to_delete, axis=1, inplace=True)

In [16]:
# to check the % of null values
round((pd.isnull(telecom_train_df).sum()/len(telecom_train_df.index)),2)[pd.isnull(telecom_train_df).sum()/ len(telecom_train_df.index) > 0.70].sort_values()

arpu_3g_8           0.74
night_pck_user_8    0.74
night_pck_user_7    0.74
arpu_2g_8           0.74
arpu_2g_7           0.74
fb_user_7           0.74
arpu_3g_7           0.74
fb_user_8           0.74
count_rech_3g_8     0.74
count_rech_3g_7     0.74
count_rech_2g_8     0.74
count_rech_2g_7     0.74
arpu_2g_6           0.75
count_rech_3g_6     0.75
night_pck_user_6    0.75
fb_user_6           0.75
arpu_3g_6           0.75
count_rech_2g_6     0.75
dtype: float64

In [17]:
# count of missing data > 70% columns in train dataset
len(round((pd.isnull(telecom_train_df).sum()/len(telecom_train_df.index)),2)[pd.isnull(telecom_train_df).sum()/ len(telecom_train_df.index) > 0.70].sort_values())

18

In [18]:
# count of missing data > 70% columns in test dataset
len(round((pd.isnull(telecom_test_df).sum()/len(telecom_test_df.index)),2)[pd.isnull(telecom_test_df).sum()/ len(telecom_test_df.index) > 0.70].sort_values())

18

### As we have more than 70% data as null for 18 columns in train and test, we are dropping these columns from dataset.

In [19]:
cols_to_delete = ((pd.isnull(telecom_train_df).sum()/len(telecom_train_df.index))[pd.isnull(telecom_train_df).sum()/ len(telecom_train_df.index) > 0.70].sort_values()).keys()

In [20]:
cols_to_delete = cols_to_delete.to_list()
print(cols_to_delete)

['arpu_3g_8', 'night_pck_user_8', 'arpu_2g_8', 'count_rech_3g_8', 'fb_user_8', 'count_rech_2g_8', 'arpu_3g_7', 'fb_user_7', 'arpu_2g_7', 'night_pck_user_7', 'count_rech_2g_7', 'count_rech_3g_7', 'count_rech_3g_6', 'arpu_3g_6', 'arpu_2g_6', 'night_pck_user_6', 'fb_user_6', 'count_rech_2g_6']


In [21]:
telecom_train_df = telecom_train_df.drop(cols_to_delete, axis=1)

In [22]:
telecom_train_df.shape

(69999, 144)

In [23]:
telecom_test_df = telecom_test_df.drop(cols_to_delete, axis=1)

In [24]:
telecom_test_df.shape

(30000, 143)

## Lets check the data types of columns now

In [25]:
telecom_train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 69999 entries, 0 to 69998
Columns: 144 entries, circle_id to churn_probability
dtypes: float64(117), int64(27)
memory usage: 77.4 MB


In [26]:
telecom_test_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30000 entries, 0 to 29999
Columns: 143 entries, circle_id to jun_vbc_3g
dtypes: float64(117), int64(26)
memory usage: 33.0 MB


### We can see now we don't have any categorical data, all are numerical data only.

### Imputing the na values with
 - Median for numerical variables
 - Mode for categorical variables - Not needed in out this case.

In [27]:
# A generic method to impute na
def replace_na(df, columns, data_type):
    for col in columns:
        if data_type == 'categorical':
            value = pd.to_datetime(df[col]).mode()
        else:
            value = df[col].median()
        df[col].fillna(value, inplace=True)
    return df

In [28]:
telecom_train_df = replace_na(telecom_train_df, telecom_train_df.columns, 'numerical')
telecom_test_df = replace_na(telecom_test_df, telecom_test_df.columns, 'numerical')

### Lets check for unique entries in train and test dataset

In [29]:
def cols_to_delete_unique(df, df_columns, threshold=4):
    # threshold=4 means column has only 1 unique value
    # eg. telecom_train_df['circle_id'].describe().unique() --> array([30000.,   109.,     0.])
    # Here because there are all 109 in column, we have only 3 entries in array.
    cols_to_delete = []
    for column in df_columns:
        if len(df[column].describe().unique()) < threshold:
            cols_to_delete.append(column)
    
    return cols_to_delete

In [30]:
print(telecom_train_df.shape)
print(telecom_test_df.shape)

(69999, 144)
(30000, 143)


In [31]:
# on train data
telecom_train_df = telecom_train_df.drop(cols_to_delete_unique(telecom_train_df, telecom_train_df.columns), axis=1)
print(telecom_train_df.shape)

# on test data
telecom_test_df = telecom_test_df.drop(cols_to_delete_unique(telecom_test_df, telecom_test_df.columns), axis=1)
print(telecom_test_df.shape)

(69999, 134)
(30000, 133)


### Lets find the high value customers based on recharge done in 6th and 7th month and update dataframe

### For train data

In [32]:
# total data recharge in 6th, 7th and 8th months
telecom_train_df["total_data_recharge_amnt_6"] = telecom_train_df.total_rech_data_6 * telecom_train_df.av_rech_amt_data_6
telecom_train_df["total_data_recharge_amnt_7"] = telecom_train_df.total_rech_data_7 * telecom_train_df.av_rech_amt_data_7
telecom_train_df["total_data_recharge_amnt_8"] = telecom_train_df.total_rech_data_8 * telecom_train_df.av_rech_amt_data_8

# total amount spent on recharge in 6th, 7th and 8th months
telecom_train_df["total_recharge_amnt_6"] = telecom_train_df.total_rech_amt_6 + telecom_train_df.total_data_recharge_amnt_6
telecom_train_df["total_recharge_amnt_7"] = telecom_train_df.total_rech_amt_7 + telecom_train_df.total_data_recharge_amnt_7
telecom_train_df["total_recharge_amnt_8"] = telecom_train_df.total_rech_amt_8 + telecom_train_df.total_data_recharge_amnt_8

# average recharge for 6th and 7th month
telecom_train_df['average_amnt_6_7'] = (telecom_train_df["total_recharge_amnt_6"] + telecom_train_df["total_recharge_amnt_7"])/2

# 70th percentile of average_amnt_6_7
telecom_train_df['average_amnt_6_7'].quantile(.70)

477.5

## Filter dataset based on average_amnt_6_7 (70th percentile for train dataset is 477.5)

In [33]:
telecom_train_df = telecom_train_df[telecom_train_df["average_amnt_6_7"]>= telecom_train_df["average_amnt_6_7"].quantile(.70)]

In [34]:
telecom_train_df.shape

(21013, 141)

## Lets visualize the 8th month and average_amnt_6_7 months data to check behavior pattern of customers

In [35]:
# Look it later
#plt.scatter(telecom_train_df[['average_amnt_6_7', 'total_recharge_amnt_8']])
#plt.show()
#sns.
#telecom_train_df.plot.scatter(x='average_amnt_6_7', y='total_recharge_amnt_8')
#telecom_train_df.DataFrame(np.random.rand(10, 3), columns =['average_amnt_6_7', 'total_recharge_amnt_8'])

#telecom_train_df.plot.bar(x="average_amnt_6_7", y="total_recharge_amnt_8", rot=70);

#ig, ax = plt.subplots()

#ax.plot(telecom_train_df['average_amnt_6_7'], telecom_train_df['total_recharge_amnt_8'])
#plt.show()

### For test data

In [36]:
# total data recharge in 6th, 7th and 8th months
telecom_test_df["total_data_recharge_amnt_6"] = telecom_test_df.total_rech_data_6 * telecom_test_df.av_rech_amt_data_6
telecom_test_df["total_data_recharge_amnt_7"] = telecom_test_df.total_rech_data_7 * telecom_test_df.av_rech_amt_data_7
telecom_test_df["total_data_recharge_amnt_8"] = telecom_test_df.total_rech_data_8 * telecom_test_df.av_rech_amt_data_8

# total amount spent on recharge in 6th, 7th and 8th months
telecom_test_df["total_recharge_amnt_6"] = telecom_test_df.total_rech_amt_6 + telecom_test_df.total_data_recharge_amnt_6
telecom_test_df["total_recharge_amnt_7"] = telecom_test_df.total_rech_amt_7 + telecom_test_df.total_data_recharge_amnt_7
telecom_test_df["total_recharge_amnt_8"] = telecom_test_df.total_rech_amt_8 + telecom_test_df.total_data_recharge_amnt_8

# average recharge
telecom_test_df['average_amnt_6_7'] = (telecom_test_df["total_recharge_amnt_6"] + telecom_test_df["total_recharge_amnt_7"])/2

# 70th percentile of average_amnt_6_7
telecom_test_df['average_amnt_6_7'].quantile(.70)

478.0

### Filter dataset based on average_amnt_6_7 (70th percentile for test dataset is 478.0)

In [37]:
telecom_test_df = telecom_test_df[telecom_test_df["average_amnt_6_7"]>= telecom_test_df["average_amnt_6_7"].quantile(.70)]
telecom_test_df.shape

(9003, 140)

## Lets focus on Outliers now and treat them

In [38]:
# Generic method to remove outliers
def remove_outliers(df, features):
    for feature in features:
        q1 = df[feature].quantile(0.25)
        q3 = df[feature].quantile(0.99)
        iqr = q3-q1
        lower_value  = q1 - (1.5 * iqr)
        higer_value = q3 + (1.5 * iqr)
        df = df[(df[feature] <= higer_value) & (df[feature] >= lower_value)]
    return df

In [39]:
#telecom_train_df = remove_outliers(telecom_train_df, telecom_train_df.columns)
#telecom_test_df = remove_outliers(telecom_test_df, telecom_test_df.columns)

In [40]:
telecom_train_df.shape

(21013, 141)

In [41]:
telecom_test_df.shape

(9003, 140)

## Let's check for Data imbalance here

In [42]:
churn_data_rows = telecom_train_df['churn_probability'].sum()

In [43]:
churn_data_rows/len(telecom_train_df) * 100

8.275829248560415

### We can see that there is a data imbalance. We have only about 8.28% of data as Churn and 91.72% as not churn.
### So we need to apply data imbalance technique. We will use SMOTE.

## Dividing the data from telecom_train_df to train the model

In [44]:
y = telecom_train_df['churn_probability']
X = telecom_train_df.drop(['churn_probability'], axis=1)

telecom_train_df.drop('churn_probability', axis=1, inplace=True)

In [45]:
# Splitting the data into test and train
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, test_size=0.3, random_state=100)

## Taking a backup of variables

In [46]:
X_train_ori, X_test_ori, y_train_ori, y_test_ori = X_train.copy(), X_test.copy(), y_train.copy(), y_test.copy()

## Rescaling of the variables

- We will use Min-Max scaling (Normalization) --> compresses all the data between 0 and 1

In [47]:
# Creating MinMaxScaler object
scaler = MinMaxScaler()
# fit on data
# removed churn_probability becuase test data is not having it.
# Also churn_probability is having 0 and 1 only. So it need not be scaled.
train_numerical_columns = telecom_train_df.columns.to_list()
X_train[train_numerical_columns] = scaler.fit_transform(X_train[train_numerical_columns])
X_train.head()
# So all numberic values are now between 0 and 1

,arpu_6,arpu_7,arpu_8,onnet_mou_6,onnet_mou_7,onnet_mou_8,offnet_mou_6,offnet_mou_7,offnet_mou_8,roam_ic_mou_6,...,aug_vbc_3g,jul_vbc_3g,jun_vbc_3g,total_data_recharge_amnt_6,total_data_recharge_amnt_7,total_data_recharge_amnt_8,total_recharge_amnt_6,total_recharge_amnt_7,total_recharge_amnt_8,average_amnt_6_7
6968,0.127277,0.274576,0.081830,0.004624,0.004018,0.002985,0.011537,0.040985,0.006554,0.000000,...,0.000000,0.002051,0.000000,0.000000,0.000000,0.000000,0.000000,0.096264,0.000000,0.069171
42203,0.156166,0.094305,0.083632,0.066057,0.041462,0.074009,0.065428,0.053213,0.049054,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.003787,0.009557,0.009933,0.016496,0.002008
18406,0.151447,0.122733,0.087708,0.015153,0.016084,0.008306,0.056327,0.259820,0.136117,0.005055,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.008632,0.019367,0.017273,0.009450
21455,0.171177,0.128305,0.077781,0.009636,0.010909,0.006579,0.081607,0.121132,0.038009,0.005166,...,0.029661,0.043865,0.039021,0.000000,0.011038,0.011124,0.015034,0.032992,0.023503,0.026840
66143,0.138174,0.087866,0.074838,0.000000,0.006171,0.001633,0.000000,0.033354,0.020053,0.000000,...,0.098218,0.000000,0.000000,0.043938,0.000000,0.016759,0.045971,0.006063,0.026593,0.029892


In [48]:
from imblearn.over_sampling import SMOTE ## Comments to be fixed 

In [49]:
X_resampled, y_resampled = SMOTE().fit_resample(X_train, y_train)

In [50]:
X_resampled.shape

(27020, 140)

In [51]:
sum(y_resampled)/len(y_resampled) * 100

50.0

## Now there is no data imbalance.

In [52]:
# Lets fit the scaler for train data as well
test_numerical_columns = telecom_test_df.columns.to_list()
telecom_test_df[test_numerical_columns] = scaler.transform(telecom_test_df[test_numerical_columns])
telecom_test_df.head()

,arpu_6,arpu_7,arpu_8,onnet_mou_6,onnet_mou_7,onnet_mou_8,offnet_mou_6,offnet_mou_7,offnet_mou_8,roam_ic_mou_6,...,aug_vbc_3g,jul_vbc_3g,jun_vbc_3g,total_data_recharge_amnt_6,total_data_recharge_amnt_7,total_data_recharge_amnt_8,total_recharge_amnt_6,total_recharge_amnt_7,total_recharge_amnt_8,average_amnt_6_7
1,0.150615,0.099685,0.064554,0.010236,0.005052,0.001845,0.056723,0.092260,0.028195,0.000000,...,0.000000,0.000000,0.0000,0.0,0.000000,0.000000,0.009067,0.008917,0.008442,0.000701
2,0.145864,0.095230,0.083592,0.001022,0.000963,0.000781,0.002031,0.006797,0.003215,0.008728,...,0.040694,0.082745,0.0319,0.0,0.000000,0.000000,0.009067,0.017833,0.016884,0.008485
21,0.152781,0.091615,0.065131,0.151258,0.103391,0.064113,0.001799,0.004487,0.001984,0.000000,...,0.000000,0.000000,0.0000,0.0,0.000000,0.000000,0.011352,0.008845,0.007362,0.002600
23,0.190866,0.119009,0.094514,0.053212,0.032421,0.040272,0.024393,0.048810,0.029353,0.012276,...,0.000000,0.000000,0.0000,0.0,0.001816,0.000000,0.022596,0.019973,0.019417,0.021967
26,0.135918,0.117934,0.085279,0.017062,0.231259,0.138588,0.002456,0.017742,0.004589,0.007378,...,0.014353,0.000000,0.0000,0.0,0.000000,0.018315,0.004715,0.014516,0.032468,0.001853


## Modelling 

1. Logistic Regression using RFE - To get important predictors for Churn probability
2. Logistic Regression - To know Churn probability + PCA
    - Lasso
    - Ridge
3. Tree models
    - XGBoost
    - Random Forest

## 1. Logistic Regression using RFE - To get importnat predictors for Churn probability

In [53]:
# For RFE
logml = sm.GLM(y_resampled, (sm.add_constant(X_resampled)), family=sm.families.Binomial())
logml.fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:      churn_probability   No. Observations:                27020
Model:                            GLM   Df Residuals:                    26886
Model Family:                Binomial   Df Model:                          133
Link Function:                  logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -8649.2
Date:                Fri, 16 Sep 2022   Deviance:                       17298.
Time:                        11:18:20   Pearson chi2:                 7.74e+04
No. Iterations:                   100                                         
Covariance Type:            nonrobust                                         
==============================================================================================
                                 coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------
const                         -3.3280      0.680     -4.892      0.000      -4.661      -1.995
arpu_6                         7.0058      3.697      1.895      0.058      -0.240      14.252
arpu_7                        17.2132      4.388      3.923      0.000       8.612      25.814
arpu_8                        40.2898      4.920      8.189      0.000      30.647      49.933
onnet_mou_6                 -127.9130     35.678     -3.585      0.000    -197.842     -57.984
onnet_mou_7                  -38.2049     31.941     -1.196      0.232    -100.807      24.397
onnet_mou_8                   24.4840     50.811      0.482      0.630     -75.104     124.072
offnet_mou_6                -129.9980     40.104     -3.241      0.001    -208.601     -51.395
offnet_mou_7                 -38.7925     25.497     -1.521      0.128     -88.765      11.180
offnet_mou_8                  -9.0238     64.732     -0.139      0.889    -135.896     117.849
roam_ic_mou_6                  1.3897      1.054      1.318      0.187      -0.676       3.456
roam_ic_mou_7                  1.5447      1.325      1.166      0.244      -1.051       4.141
roam_ic_mou_8                 -2.6304      1.423     -1.849      0.064      -5.419       0.158
roam_og_mou_6                 62.4488     18.227      3.426      0.001      26.725      98.172
roam_og_mou_7                 19.1128     10.786      1.772      0.076      -2.026      40.252
roam_og_mou_8                  1.6412     24.987      0.066      0.948     -47.332      50.614
loc_og_t2t_mou_6           -1.051e+04   2.54e+04     -0.414      0.679   -6.02e+04    3.92e+04
loc_og_t2t_mou_7           -1.188e+04   2.88e+04     -0.413      0.680   -6.83e+04    4.46e+04
loc_og_t2t_mou_8            2.504e+05   4.41e+04      5.684      0.000    1.64e+05    3.37e+05
loc_og_t2m_mou_6           -7685.5533   1.85e+04     -0.415      0.678    -4.4e+04    2.86e+04
loc_og_t2m_mou_7           -7314.1614   1.77e+04     -0.413      0.680   -4.21e+04    2.74e+04
loc_og_t2m_mou_8            1.156e+05   2.03e+04      5.685      0.000    7.57e+04    1.55e+05
loc_og_t2f_mou_6           -1014.0132   2435.361     -0.416      0.677   -5787.233    3759.206
loc_og_t2f_mou_7           -1308.3260   3172.187     -0.412      0.680   -7525.699    4909.047
loc_og_t2f_mou_8            1.371e+04   2410.161      5.687      0.000    8981.591    1.84e+04
loc_og_t2c_mou_6              -8.1214      1.492     -5.442      0.000     -11.047      -5.196
loc_og_t2c_mou_7              -5.2216      2.629     -1.986      0.047     -10.375      -0.069
loc_og_t2c_mou_8               7.6600      1.611      4.754      0.000       4.502      10.818
loc_og_mou_6                3.038e+04   4.78e+04      0.636      0.525   -6.33e+04    1.24e+05
loc_og_mou_7               -2.125e+04   3.5

## Lower the p-value higher the significance

In [54]:
logreg = LogisticRegression()

In [55]:
# Running rfe with 70 variables as output
rfe = RFE(logreg, n_features_to_select=70)
rfe = rfe.fit(X_resampled, y_resampled)

In [56]:
rfe.support_ # it givs whether or not the selected feature was in top 70. True means yes

array([ True,  True,  True, False,  True, False, False,  True,  True,
       False, False, False,  True,  True,  True, False, False,  True,
        True, False,  True,  True, False,  True,  True,  True,  True,
       False, False,  True, False, False, False, False, False,  True,
       False, False,  True,  True,  True,  True, False, False, False,
       False, False,  True, False, False, False, False, False,  True,
       False, False,  True,  True, False,  True, False, False,  True,
        True,  True,  True,  True, False,  True, False,  True, False,
       False, False,  True,  True, False,  True,  True, False,  True,
       False, False,  True, False, False,  True, False, False, False,
        True,  True,  True,  True, False,  True,  True,  True,  True,
       False, False,  True,  True, False,  True,  True,  True,  True,
        True, False,  True,  True,  True,  True, False, False,  True,
       False,  True,  True, False, False,  True, False, False,  True,
       False, False,

In [57]:
# to see the selected columns
list(zip(X_resampled.columns, rfe.support_, rfe.ranking_))

[('arpu_6', True, 1),
 ('arpu_7', True, 1),
 ('arpu_8', True, 1),
 ('onnet_mou_6', False, 14),
 ('onnet_mou_7', True, 1),
 ('onnet_mou_8', False, 49),
 ('offnet_mou_6', False, 12),
 ('offnet_mou_7', True, 1),
 ('offnet_mou_8', True, 1),
 ('roam_ic_mou_6', False, 2),
 ('roam_ic_mou_7', False, 59),
 ('roam_ic_mou_8', False, 58),
 ('roam_og_mou_6', True, 1),
 ('roam_og_mou_7', True, 1),
 ('roam_og_mou_8', True, 1),
 ('loc_og_t2t_mou_6', False, 56),
 ('loc_og_t2t_mou_7', False, 51),
 ('loc_og_t2t_mou_8', True, 1),
 ('loc_og_t2m_mou_6', True, 1),
 ('loc_og_t2m_mou_7', False, 33),
 ('loc_og_t2m_mou_8', True, 1),
 ('loc_og_t2f_mou_6', True, 1),
 ('loc_og_t2f_mou_7', False, 54),
 ('loc_og_t2f_mou_8', True, 1),
 ('loc_og_t2c_mou_6', True, 1),
 ('loc_og_t2c_mou_7', True, 1),
 ('loc_og_t2c_mou_8', True, 1),
 ('loc_og_mou_6', False, 60),
 ('loc_og_mou_7', False, 57),
 ('loc_og_mou_8', True, 1),
 ('std_og_t2t_mou_6', False, 35),
 ('std_og_t2t_mou_7', False, 11),
 ('std_og_t2t_mou_8', False, 50),
 (

In [58]:
col = X_resampled.columns[rfe.support_]

In [59]:
X_resampled_sm = sm.add_constant(X_resampled[col])
logm2 = sm.GLM(y_resampled, X_resampled_sm, family=sm.families.Binomial())
res = logm2.fit()
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:      churn_probability   No. Observations:                27020
Model:                            GLM   Df Residuals:                    26950
Model Family:                Binomial   Df Model:                           69
Link Function:                  logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -8882.2
Date:                Fri, 16 Sep 2022   Deviance:                       17764.
Time:                        11:18:51   Pearson chi2:                 9.79e+04
No. Iterations:                     7                                         
Covariance Type:            nonrobust                                         
======================================================================================
                         coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                 -2.1000      0.516     -4.069      0.000      -3.112      -1.088
arpu_6                 5.0001      3.329      1.502      0.133      -1.525      11.526
arpu_7                 7.2192      2.364      3.053      0.002       2.585      11.853
arpu_8                35.0273      4.409      7.945      0.000      26.387      43.668
onnet_mou_7            4.2258      1.479      2.857      0.004       1.327       7.125
offnet_mou_7           3.1414      1.209      2.598      0.009       0.772       5.511
offnet_mou_8         -44.9001      9.907     -4.532      0.000     -64.318     -25.483
roam_og_mou_6          0.8530      0.815      1.047      0.295      -0.743       2.449
roam_og_mou_7          3.6801      0.884      4.163      0.000       1.947       5.413
roam_og_mou_8         15.4298      2.959      5.215      0.000       9.631      21.228
loc_og_t2t_mou_8    6.276e+04   3.81e+04      1.648      0.099   -1.19e+04    1.37e+05
loc_og_t2m_mou_6      -2.1257      0.873     -2.436      0.015      -3.836      -0.415
loc_og_t2m_mou_8    2.897e+04   1.76e+04      1.649      0.099   -5464.798    6.34e+04
loc_og_t2f_mou_6      -3.0733      1.556     -1.975      0.048      -6.124      -0.023
loc_og_t2f_mou_8    3438.0104   2083.551      1.650      0.099    -645.674    7521.695
loc_og_t2c_mou_6      -7.1771      1.254     -5.722      0.000      -9.636      -4.719
loc_og_t2c_mou_7      -7.1608      2.037     -3.516      0.000     -11.153      -3.169
loc_og_t2c_mou_8       8.0789      1.507      5.362      0.000       5.126      11.032
loc_og_mou_8       -6.439e+04   3.91e+04     -1.647      0.100   -1.41e+05    1.22e+04
std_og_t2m_mou_8      40.5084      9.947      4.072      0.000      21.012      60.005
std_og_t2f_mou_8      -7.9486      2.191     -3.629      0.000     -12.242      -3.655
std_og_mou_6           1.5764      0.567      2.781      0.005       0.466       2.687
std_og_mou_7          -0.0996      1.475     -0.068      0.946      -2.990       2.790
std_og_mou_8          68.9274     35.157      1.961      0.050       0.022     137.833
spl_og_mou_8          -0.8173      2.592     -0.315      0.753      -5.897       4.262
total_og_mou_8       -78.5250     35.447     -2.215      0.027    -148.001      -9.049
loc_ic_t2t_mou_8   -1.505e+04   8594.434     -1.751      0.080   -3.19e+04    1797.698
loc_ic_t2m_mou_6       3.6317      1.857      1.956      0.050      -0.007       7.271
loc_ic_t2m_mou_8   -1.759e+04      1e+04     -1.751      0.080   -3.73e+04    2102.104
loc_ic_t2f_mou_8   -5532.4269   3158.537     -1.752      0.080   -1.17e+04     658.191
loc_ic_mou_6           4.2347      4.307      0.983      0.325      -4.206      12.676
loc_ic_mou_7          15.5527      1.611      9.654      0.000      12.395      18.710
loc_ic_mou_8        2.006e+04   1.15e+

### Making predictions on the test set

In [60]:
X_test = X_test[col]
X_test.head()

,arpu_6,arpu_7,arpu_8,onnet_mou_7,offnet_mou_7,offnet_mou_8,roam_og_mou_6,roam_og_mou_7,roam_og_mou_8,loc_og_t2t_mou_8,...,vol_2g_mb_8,vol_3g_mb_8,monthly_2g_7,monthly_2g_8,sachet_2g_8,monthly_3g_8,sachet_3g_8,aon,aug_vbc_3g,jul_vbc_3g
5662,554.740,782.352,673.692,38.63,845.83,661.89,0.0,0.00,0.0,15.56,...,0.0,0.0,0,0,0,0,0,1360,0.00,0.0
37574,1563.157,1579.675,1256.565,3158.18,1144.44,1327.03,0.0,0.00,0.0,5.91,...,0.0,0.0,0,0,0,0,0,834,0.00,0.0
58341,594.217,796.737,996.393,1200.53,186.48,585.36,0.0,0.00,0.0,8.53,...,0.0,0.0,0,0,0,0,0,594,0.00,0.0
23282,352.160,197.245,365.564,179.88,116.58,507.64,0.0,34.96,0.0,227.84,...,0.0,0.0,0,0,3,0,0,1164,0.00,0.0
30561,332.040,275.976,267.033,96.89,178.24,424.68,0.0,0.00,0.0,3.78,...,0.0,0.0,0,0,0,0,0,635,31.05,337.7


In [61]:
X_test_sm = sm.add_constant(X_test)

In [62]:
y_test_pred = res.predict(X_test_sm)

In [63]:
y_test_pred[:10]

5662     0.0
37574    0.0
58341    0.0
23282    0.0
30561    0.0
45844    0.0
30159    0.0
62778    0.0
45559    0.0
35274    0.0
dtype: float64

In [64]:
X_test_sm.head()

,const,arpu_6,arpu_7,arpu_8,onnet_mou_7,offnet_mou_7,offnet_mou_8,roam_og_mou_6,roam_og_mou_7,roam_og_mou_8,...,vol_2g_mb_8,vol_3g_mb_8,monthly_2g_7,monthly_2g_8,sachet_2g_8,monthly_3g_8,sachet_3g_8,aon,aug_vbc_3g,jul_vbc_3g
5662,1.0,554.740,782.352,673.692,38.63,845.83,661.89,0.0,0.00,0.0,...,0.0,0.0,0,0,0,0,0,1360,0.00,0.0
37574,1.0,1563.157,1579.675,1256.565,3158.18,1144.44,1327.03,0.0,0.00,0.0,...,0.0,0.0,0,0,0,0,0,834,0.00,0.0
58341,1.0,594.217,796.737,996.393,1200.53,186.48,585.36,0.0,0.00,0.0,...,0.0,0.0,0,0,0,0,0,594,0.00,0.0
23282,1.0,352.160,197.245,365.564,179.88,116.58,507.64,0.0,34.96,0.0,...,0.0,0.0,0,0,3,0,0,1164,0.00,0.0
30561,1.0,332.040,275.976,267.033,96.89,178.24,424.68,0.0,0.00,0.0,...,0.0,0.0,0,0,0,0,0,635,31.05,337.7


In [65]:
X_resampled_sm.shape

(27020, 71)

In [66]:
y_pred_train = res.predict(X_resampled_sm)

In [67]:
y_pred_train.head()

0    0.997640
1    0.080108
2    0.117195
3    0.057178
4    0.005226
dtype: float64

In [68]:
y_pred_train.shape

(27020,)

## Confusion matrix for Precision and Recall and Accuracy of the model

In [69]:
y_resampled

0        0
1        0
2        0
3        0
4        0
        ..
27015    1
27016    1
27017    1
27018    1
27019    1
Name: churn_probability, Length: 27020, dtype: int64

In [70]:
y_pred_train_final = pd.DataFrame({'Churn_Prob': y_pred_train})

In [71]:
y_pred_train_final.shape

(27020, 1)

In [72]:
y_resampled.shape

(27020,)

In [73]:
confusion = metrics.confusion_matrix('y_resampled', y_pred_train_final)
confusion

ValueError: Found input variables with inconsistent numbers of samples: [11, 27020]

In [ ]:
metric = []
r2_train_lr = r2_score(y_resampled, y_pred_train)
print(f"r-squared train: {r2_train_lr}")
metric.append(r2_train_lr)

rss1_lr = np.sum(np.square(y_resampled - y_pred_train))
print(f"rss train: {rss1_lr}")
metric.append(rss1_lr)

mse_train_lr = mean_squared_error(y_resampled, y_pred_train)
print(f"mse train: {mse_train_lr}")
metric.append(mse_train_lr**0.5)

## Looking at the rmse value (close to 0.1) Logistic regression doesn't seems to be a good model with such large number of  columns.

## So lets move on with PCA and Logistic Regression to check if model improves or not.

## PCA on the data

- While computing the principal components, we must not include the entire dataset. Model building is all about doing well on the data we haven't seen yet!
- So we'll calculate the PCs using the train data, and apply them later on the test data.

In [ ]:
# Looking at the data again

In [ ]:
y_resampled.shape

In [ ]:
X_resampled.shape

In [ ]:
pca = PCA(random_state=42)

In [ ]:
pca.fit(X_resampled)

In [ ]:
pca.components_

In [ ]:
pca.explained_variance_ratio_

In [ ]:
var_cumu = np.cumsum(pca.explained_variance_ratio_)

In [ ]:
fig = plt.figure(figsize=[12,8])
plt.plot(range(1,len(var_cumu)+1), var_cumu)

## Looking at the scree plot with 50 PCs we have more than 95% of variance explained.

In [ ]:
pca_final = IncrementalPCA(n_components=50)

In [ ]:
df_train_pca = pca_final.fit_transform(X_resampled)

In [ ]:
df_train_pca.shape

In [ ]:
corrmat = np.corrcoef(df_train_pca.transpose())

In [ ]:
corrmat.shape

In [ ]:
corrmat

### Applying the transformation on the test set

In [ ]:
X_test.shape

In [ ]:
df_test_pca = pca_final.transform(X_test_ori)
df_test_pca.shape

## Applying logistic regression on the data on our Principal components

In [ ]:
learner_pca = LogisticRegression()

In [ ]:
model_pca = learner_pca.fit(df_train_pca, y_resampled)

### Making predictions on the test set

In [ ]:
pred_probs_test = model_pca.predict_proba(df_test_pca)

In [ ]:
"{:2.2}".format(metrics.roc_auc_score(y_test, pred_probs_test[:,1]))

##### So we could see the accuracy coming to 80%.
We got a rough idea with PCA and Logistic Regression.
We now move to Lasso and Ridge Regression for further analysis and actual conclusion on right predictors and coefficients.
Let's try other models to see how it performs.

### Ridge and Lasso Regression


#### Ridge Regression

In [ ]:
# list of alphas to tune - if value too high it will lead to underfitting, if it is too low, 
# it will not handle the overfitting
params = {'alpha': [0.0001, 0.001, 0.01, 0.05, 0.1, 
 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 2.0, 3.0, 
 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 20, 50, 100, 500, 1000 ]}

ridge = Ridge()

# cross validation
folds = 10
model_cv = GridSearchCV(estimator = ridge, 
                        param_grid = params, 
                        scoring= 'neg_mean_absolute_error',  
                        cv = folds, 
                        return_train_score=True,
                        verbose = 1)  


model_cv.fit(X_resampled, y_resampled)

In [ ]:
# Printing the best hyperparameter alpha
print(model_cv.best_params_)

In [ ]:
#Fitting Ridge model for alpha = 0.0001 and printing coefficients which have been penalised
alpha = 0.0001

ridge = Ridge(alpha=alpha)
        
ridge.fit(X_resampled, y_resampled) 

In [ ]:
cv_result_r = pd.DataFrame(model_cv.cv_results_)
cv_result_r['param_alpha'] = cv_result_r['param_alpha'].astype('float32')
cv_result_r.head()

In [ ]:
plt.figure(figsize=(16,8))
plt.plot(cv_result_r['param_alpha'],cv_result_r['mean_train_score'])
plt.plot(cv_result_r['param_alpha'],cv_result_r['mean_test_score'])
plt.xscale('log')
plt.ylabel('R2 Score')
plt.xlabel('Alpha')
plt.show()

## Looking at the curve, as they are not converging we will go ahead with Lasso.

In [ ]:
# Lets do prediction for test data and train data
# train data

In [ ]:
y_train_pred = ridge.predict(X_resampled)
metrics.r2_score(y_true=y_resampled, y_pred=y_train_pred)

In [ ]:
# Prediction for test data 
y_pred_test = ridge.predict(X_resampled)

In [ ]:
y_pred_test